In [90]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from genetic_algorithm import GeneticAlgorithm

# Load the data
data = pd.read_csv(r"C:\Users\Karthick NG\Desktop\KARTHICK\SNU\4TH SEMESTER\1.ML\1.ML LAB\EX_9\Bank_Personal_Loan_Modelling.csv")


In [91]:

X = data.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)
y = data['Personal Loan']
X = (X - X.mean()) / X.std()


In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [93]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score




In [94]:
import numpy as np
from sklearn.metrics import log_loss

class AntColonyOptimizer:
    def __init__(self, model, X, y, ant_count, max_iter, q, alpha, beta, rho):
        self.model = model
        self.X = X
        self.y = y
        self.ant_count = ant_count
        self.max_iter = max_iter
        self.q = q
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.tau = np.ones((self.X.shape[1],))




In [95]:
    def _select_next_feature(self, feature_set, feature_probabilities):
        # Select the next feature to add to the feature set
        feature_idx = np.random.choice(np.arange(self.X.shape[1]), p=feature_probabilities)
        feature_set.add(feature_idx)
        # Update the pheromone trail for the selected feature
        self.tau[feature_idx] *= (1 - self.rho)
        self.tau[feature_idx] += self.rho * self.q
        return feature_idx



In [96]:
# # Initialize the ACO optimizer
# aco = AntColonyOptimizer(model, X_train, y_train, ant_count=10, max_iter=100, q=1, alpha=1, beta=2, rho=0.5)



In [97]:
    def _calculate_probabilities(self, feature_set):
        # Calculate the probabilities of selecting each feature not in the feature set
        unselected_features = np.delete(np.arange(self.X.shape[1]), list(feature_set))
        probabilities = np.zeros((len(unselected_features),))
        tau_set = self.tau[list(feature_set)]
        denominator = np.sum(tau_set * np.power(np.abs(self.X[:, unselected_features] - self.X[:, feature_set]), self.alpha) * np.power(np.abs(self.y - self.model.predict(self.X[:, feature_set]).ravel()), self.beta))
        for i, feature_idx in enumerate(unselected_features):
            numerator = self.tau[feature_idx] * np.power(np.abs(self.X[:, feature_idx] - self.X[:, feature_set]), self.alpha) * np.power(np.abs(self.y - self.model.predict(np.column_stack([self.X[:, list(feature_set)], self.X[:, feature_idx]])).ravel()), self.beta)
            probabilities[i] = numerator / denominator
        return probabilities


In [98]:

    def optimize_weights(self):
        # Perform feature selection using ACO
        best_feature_set = set()
        best_loss = float('inf')
        for iteration in range(self.max_iter):
            feature_set = set()
            feature_probabilities = np.ones((self.X.shape[1],))
            while len(feature_set) < int(np.ceil(self.X.shape[1] * 0.5)):
                # Select the next feature to add to the feature set
                feature_idx = self._select_next_feature(feature_set, feature_probabilities)
                # Update the probabilities based on the new feature
                feature_probabilities = self._calculate_probabilities(feature_set)
                feature_probabilities /= feature_probabilities.sum()  # normalize probabilities
            # Train a model using the selected features and calculate the log loss
            selected_features = sorted(list(feature_set))
            self.model.fit(self.X[:, selected_features], self.y, epochs=10, batch_size=32, verbose=0)
            loss = log_loss(self.y, self.model.predict(self.X[:, selected_features]))
            # Update the best feature set if the log loss is lower
            if loss < best_loss:
                best_feature_set = feature_set
                best_loss = loss
        # Set the best weights to the model
        selected_features = sorted(list(best_feature_set))
        best_weights = self.model.get_weights()
        for i, feature_idx in enumerate(selected_features):
            best_weights[0][:, i] *= np.abs(self.X[:, feature_idx])
        self.model.set_weights(best_weights)

In [99]:

# # Optimize the weights using ACO
# best_weights = aco.optimize_weights()


In [100]:

# # Set the best weights to the model
# model.set_weights(best_weights)

# # Evaluate the model on the test set
# accuracy = accuracy_score(y_test, model.predict(X_test).round())
# print('Test accuracy:', accuracy)

In [101]:


# class AntColonyOptimizer:
#     def __init__(self, model, X_train, y_train, ant_count=10, max_iter=100, q=1, alpha=1, beta=2, rho=0.5):
#         self.model = model
#         self.X_train = X_train
#         self.y_train = y_train
#         self.ant_count = ant_count
#         self.max_iter = max_iter
#         self.q = q
#         self.alpha = alpha
#         self.beta = beta
#         self.rho = rho
#         self.pheromone = np.ones((X_train.shape[1],))
#         self.best_weights = None
#         self.best_accuracy = 0.0
        
#     def _compute_probabilities(self, weight_index, selected_features):
#         unselected_features = np.setdiff1d(range(self.X_train.shape[1]), selected_features)
#         numerators = (self.pheromone[weight_index, f]**self.alpha) * ((1 / self.model.evaluate(self.X_train[:, selected_features + [f]], self.y_train, verbose=0)[0])**self.beta)
#         denominators = np.sum((self.pheromone[weight_index, unselected_features]**self.alpha) * ((1 / self.model.evaluate(self.X_train[:, selected_features + [f]], self.y_train, verbose=0)[0])**self.beta))
#         probabilities = numerators / denominators
#         return probabilities
    
#     def _generate_solution(self):
#         selected_features = []
#         for weight_index in range(self.X_train.shape[1]):
#             probabilities = self._compute_probabilities(weight_index, selected_features)
#             selected_feature = np.random.choice(range(self.X_train.shape[1]), p=probabilities)
#             selected_features.append(selected_feature)
#         return selected_features
    


#     def _evaluate_solution(self, selected_features):
#         accuracy = accuracy_score(self.y_train, self.model.predict(self.X_train[:, selected_features]).round())
#         if accuracy > self.best_accuracy:
#             self.best_accuracy = accuracy
#             self.best_weights = self.model.get_weights()
#         return accuracy
        
#     def _update_pheromone(self):
#         for i in range(self.ant_count):
#             selected_features = self._generate_solution()
#             accuracy = self._evaluate_solution(selected_features)
#             for weight_index in range(self.X_train.shape[1]):
#                 if weight_index in selected_features:
#                     self.pheromone[weight_index] += self.q * accuracy
#                 else:
#                     self.pheromone[weight_index] *= (1 - self.rho)
                    
#     def optimize_weights(self):
#         for i in range(self.max_iter):
#             self._update_pheromone()
#         self.model.set_weights(self.best_weights)
#         return self.best_weights
        

# # Load data
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5, random_state=42)

# # Split into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create the model
# model = Sequential()
# model.add(Dense(10, input_dim=20, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])